### **Imports**

In [4]:
import time
start_time = time.time()
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import io
import string
import os

### **DataSet** **Upload**

In [5]:
from google.colab import files , drive

#upload for files in google drive
drive.mount('/content/gdrive')
dataset = "/content/gdrive/My Drive/Colab Notebooks/covid_data.csv"

'from google.colab import files , drive\n\n#upload for files in google drive\ndrive.mount(\'/content/gdrive\')\ndataset = "/content/gdrive/My Drive/Colab Notebooks/covid_data.csv"\n'

In [9]:
"""
#alternative for local file upload (file needs to be downloaded locally)
from google.colab import files
uploaded = files.upload()
df = pd.read_csv(io.BytesIO(uploaded['covid_data.csv']))
"""

Saving covid_data.csv to covid_data.csv


KeyboardInterrupt: ignored

### **DataUpload check**

In [ ]:
df = pd.read_csv(dataset)

In [ ]:
df.shape

In [ ]:
df.head(5)

In [ ]:
df.nunique()

In [ ]:
df.dtypes.value_counts()

In [ ]:
df.dtypes

### **Data Preprocessing**

In [ ]:
df.drop(['new_cases_smoothed','new_deaths_smoothed','new_cases_smoothed_per_million','new_deaths_smoothed_per_million',
         'new_tests_smoothed_per_thousand','new_tests_smoothed', 'new_vaccinations_smoothed', 'new_vaccinations_smoothed_per_million'], axis=1, inplace=True)

In [ ]:
df = df[df.continent == 'Europe'] #filters europe only entries

In [ ]:
df.continent.value_counts() #reduced entries from 67K to 16K

In [ ]:
"""
Loop to select the countries with the most common entries
INPUT: series of countries with number of entries
OUTPUT: list of the most common countries with the number of entries
"""

i = 0
theMostCommonCountry_List = []
while i < 10:
  part_index = str(df.location.value_counts().index[i])
  theMostCommonCountry_List.append( part_index )
  i = i + 1
print(theMostCommonCountry_List)

In [ ]:
df = df[df.location.isin(theMostCommonCountry_List)]  #filters entries by list of common countries

In [ ]:
df = df.reset_index()   #resets index
df.drop(columns=['index','continent'], inplace=True) #drops old index column and continent becasue it is not needed
df

In [ ]:
"""
Loop for checking which features are object type
INPUT: pandas series
OUTPUT: list of objects
"""
list_of_objects = []
x=0
while x < len(df.columns):
  if df.dtypes.values[x] == object:
    list_of_objects.append(df.dtypes.index[x])
  x = x + 1
print(list_of_objects)

In [ ]:
df.location.value_counts()

In [ ]:
correlation_matrix = df.corr().round(2)
correlation_matrix.shape

In [ ]:
#df.corr() #Tried to visualise the matrix

In [ ]:
df.isna().sum()

In [ ]:
df = df.drop(columns=['handwashing_facilities','weekly_icu_admissions','weekly_icu_admissions_per_million', 'weekly_hosp_admissions', 'weekly_hosp_admissions_per_million'])

In [ ]:
#df.groupby(['date', 'people_vaccinated','location'])['date'].count()

In [ ]:
"""while i < len(df.stringency_index):
  if x != df.location[i]:
    print("\n")
  if df.stringency_index.isna()[i] == True:
    print(df.stringency_index.isna()[i], df.location[i], df.date[i])
  i = i + 1"""

In [ ]:
df.keys()[i]

In [ ]:
train_set_X = ['total_tests', 'new_tests', 'total_deaths', 'new_deaths', 'reproduction_rate', 'positive_rate', 'population_density']

df.columns.value_counts().sum()

In [ ]:
i = 0
num_of_columns = len(df.columns)
while i < num_of_columns:
  if df.columns[i] not in ['total_tests', 'new_tests', 'total_deaths', 'new_deaths', 'reproduction_rate', 'positive_rate', 'population_density']:
    print(i, "      ", df.columns[i])
    df.drop(columns=[df.columns[i]], inplace=True)
  i = i + 1